In [ ]:
import torch
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO

# 选择预训练的DeepLabV3模型
model = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=True)
model.eval()

# 图像预处理函数
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图像转为Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 用ImageNet的均值和标准差进行归一化
])

# 获取测试图像（你可以替换为自己本地的图像）
url = "https://example.com/your-image.jpg"  # 这里换成你想测试的图像URL
response = requests.get(url)
img = Image.open(BytesIO(response.content))

# 显示原图
plt.imshow(img)
plt.title("Original Image")
plt.show()

# 进行图像预处理
input_tensor = transform(img).unsqueeze(0)  # 增加一个batch维度

# 使用GPU加速（如果有的话）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
input_tensor = input_tensor.to(device)

# 进行预测
with torch.no_grad():
    output = model(input_tensor)["out"][0]  # 获取预测结果
    output_predictions = output.argmax(0)  # 获取每个像素的类别预测

# 可视化分割结果
plt.imshow(output_predictions.cpu().numpy())
plt.title("Segmented Image")
plt.show()
